# Analysis of the developer community based on a study by Stack Overflow
### Introduction
The presented analysis is based on data obtained as a result of a survey conducted by Stack Overflow - the most popular discussion forum for programmers. The dataset contains nearly 65,000 observations on developers from around the world. This information includes their age, form of employment, technologies they work with, and their views on the labor market.

Dataset used in the study: https://stackoverflow.blog/2020/07/27/public-data-release-of-stack-overflows-2020-developer-survey/

The study, after previous corrections, also used a set with codes in the ISO 3166-1 standard for countries: https://gist.github.com/tadast/8827699

If you want to run it, use Databricks Communiuty Edition. 

This file would show you the basics of pyspark and Hive.

In [0]:
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, HiveContext
from pyspark.sql.types import *
from pyspark.sql.window import Window


d=spark.read\
  .format('com.databricks.spark.csv')\
  .options(inferSchema="true", header="true", delimiter=",")\
  .load("/FileStore/tables/survey_results_public.csv")

country_codes=spark.read\
  .format('com.databricks.spark.csv')\
  .options(inferSchema="true", header="true", delimiter=",")\
  .load("/FileStore/tables/cc.csv")\
  .withColumnRenamed("Country","cc_country").withColumnRenamed("Alpha-3 code","cc_code")\
  .select("cc_country","cc_code")\
  .withColumn('cc_code', regexp_replace('cc_code', '"', ''))\
  .withColumn('cc_code', regexp_replace('cc_code', ' ', ''))


d=d\
  .withColumn("YearStampCodePro",when(col("YearsCodePro").isin(*range(1,6,1)),"1 - 5 years")\
             .when(col("YearsCodePro") == "Less than 1 year","0 - 1 year")\
             .when(col("YearsCodePro").isin(*range(6,11,1)),"6  - 10 years")\
             .when(col("YearsCodePro").isin(*range(11,21,1)),"11 - 20 years")\
             .when(col("YearsCodePro").isin(*range(21,31,1)),"21 - 30 years")\
             .when(col("YearsCodePro").isin(*range(31,51,1)),"31 - 50 years")\
             .when(col("YearsCodePro").isin(*range(31,51,1)),"31 - 50 years")\
             .when(col("YearsCodePro")== "More than 50 years","51 or more years")\
             .otherwise(col("YearsCodePro")))

d.join(country_codes,d.Country==country_codes.cc_country,"left").write.saveAsTable("SO_joined")

## Short SQL demonstration

In [0]:
%sql
alter table so_joined
add columns (profDev string);

update SO_joined
set profDev  = ( 
case
when MainBranch="I am a developer by profession" then "Proffesional" 
when MainBranch="NA" then "NA"
else "Not proffesional"
end);

num_affected_rows
64461


## Getting a DF from database

In [0]:
df=spark.sql('SELECT * FROM SO_joined')

top10=df.groupBy("Country").count().orderBy("count",ascending=False)
top10=[row.Country for row in top10.collect()][:10]

## Functions that would make reporting easier

In [0]:
def col_array(colname,colname2=0,*argsv):
  if colname2==0:
    a=df.select(colname,*argsv).withColumn(colname,split(df[colname],';').cast(ArrayType(StringType()))).select(explode(col(colname)),*argsv)
  else:
    a=df.where(col(colname)!="NA").where(col(colname2)!="NA").select(colname,*argsv).withColumn(colname,split(df[colname],';').cast(ArrayType(StringType()))).select(explode(col(colname)),*argsv)
  return a

def NotNACount(colname,colname2=0):
  if colname2==0:
    a=df.where(col(colname)!='NA').count()
  else:
    a=df.where(col(colname)!='NA').where(col(colname2)!='NA').count()
  return a

def ArrColPercent(colname,colname2=0):
  if colname2==0:
    a=col_array(colname).where(col("col")!="NA").groupBy("col").count()\
  .withColumn('Percent',round(col("count")/NotNACount(colname)*100,1))\
  .withColumnRenamed("col",colname)\
  .orderBy("Percent",ascending=False)
  else:
    a=col_array(colname,colname2).where(col("col")!="NA").groupBy("col").count()\
  .withColumn('Percent',round(col("count")/NotNACount(colname,colname2)*100,1))\
  .withColumnRenamed("col",colname)\
  .orderBy("Percent",ascending=False)
  return a

def YearComp(CurrCol,NextCol):
  a= ArrColPercent(CurrCol,NextCol).withColumn("Year",lit("Current Year")).union(ArrColPercent(NextCol,CurrCol).withColumn("Year",lit("Next Year")))
  return a

## Count of respondents by country

In [0]:
#Wyłączenie Kosowa, które nie jest rejestrowane przez standardy ISO
mapa = df.where(df.cc_code!="XKX").groupBy("cc_code","cc_country").count().na.drop()
mapa.display()

cc_code,cc_country,count
HTI,Haiti,5
AUT,Austria,539
FSM,"Micronesia, Federated States of...",1
ECU,Ecuador,49
JAM,Jamaica,14
RWA,Rwanda,31
UZB,Uzbekistan,41
MEX,Mexico,540
DOM,Dominican Republic,72
COL,Colombia,285


##Gender

In [0]:
ArrColPercent("Gender").display()

Gender,count,Percent
Man,46236,91.5
Woman,4038,8.0
"Non-binary, genderqueer, or gender non-conforming",624,1.2


## Main Branch

In [0]:
display(df.where(col("MainBranch")!='NA').groupBy("MainBranch").count())

MainBranch,count
"I am not primarily a developer, but I write code sometimes as part of my work",5502
I code primarily as a hobby,2314
I am a developer by profession,47193
I am a student who is learning to code,7970
"I used to be a developer by profession, but no longer am",1183


## Average age by country abd developer type

In [0]:
display(df.where(col("Age")!='NA').where(col("profDev")!='NA').groupBy("Country","profDev").agg(count("Country"),round(mean("Age"),1).alias("Average Age")).orderBy("count(Country)",ascending=False).filter(df.Country.isin(top10)).withColumnRenamed("profDev","Developer Type"))

Country,Developer Type,count(Country),Average Age
United States,Proffesional,7461,34.3
India,Proffesional,3239,27.0
United Kingdom,Proffesional,2392,33.4
Germany,Proffesional,2323,32.0
United States,Not proffesional,2146,31.4
Canada,Proffesional,1291,33.0
India,Not proffesional,1171,23.2
France,Proffesional,1117,31.1
Brazil,Proffesional,1094,29.4
Netherlands,Proffesional,816,32.7


## Average age of first coding experiance by country and developer type

In [0]:
display(df.where(col("Age1stCode")!='NA').where(col("profDev")!='NA').groupBy("Country","profDev").agg(count("Country"),round(mean("Age1stCode"),1).alias("Average Age when coded for first time")).orderBy("count(Country)",ascending=False).filter(df.Country.isin(top10)).withColumnRenamed("profDev","Developer Type"))

Country,Developer Type,count(Country),Average Age when coded for first time
United States,Proffesional,9005,14.9
India,Proffesional,5032,17.1
United Kingdom,Proffesional,2859,14.4
United States,Not proffesional,2757,15.3
Germany,Proffesional,2736,14.2
India,Not proffesional,1872,16.6
Canada,Proffesional,1546,14.8
France,Proffesional,1310,15.0
Brazil,Proffesional,1283,15.8
Netherlands,Proffesional,946,14.4


##Education level

In [0]:
df.where(col("EdLevel")!="NA").groupBy("EdLevel").count().display()

EdLevel,count
Primary/elementary school,941
"Other doctoral degree (Ph.D., Ed.D., etc.)",1690
I never completed any formal education,493
"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",26542
"Associate degree (A.A., A.S., etc.)",1843
"Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",4771
"Professional degree (JD, MD, etc.)",800
"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",13112
Some college/university study without earning a degree,7239


## Developer Type

In [0]:
ArrColPercent("DevType").withColumnRenamed("DevType","Developer Type").display()

Developer Type,count,Percent
"Developer, back-end",27228,55.2
"Developer, full-stack",27125,54.9
"Developer, front-end",18296,37.1
"Developer, desktop or enterprise applications",11784,23.9
"Developer, mobile",9482,19.2
DevOps specialist,5969,12.1
Database administrator,5722,11.6
Designer,5321,10.8
System administrator,5242,10.6
"Developer, embedded applications or devices",4750,9.6


#Technologies
##Operating system

In [0]:
df.where(col("OpSys")!="NA").groupBy("OpSys").count().display()

OpSys,count
Linux-based,14932
MacOS,14075
BSD,51
Windows,27170


##Technologies today and in the future.

## Programming languages.

In [0]:
YearComp("LanguageWorkedWith","LanguageDesireNextYear").withColumnRenamed("LanguageWorkedWith","Language").display()

Language,count,Percent,Year
JavaScript,36292,67.6,Current Year
HTML/CSS,33849,63.0,Current Year
SQL,29446,54.8,Current Year
Python,23789,44.3,Current Year
Java,21578,40.2,Current Year
Bash/Shell/PowerShell,17974,33.5,Current Year
C#,16826,31.3,Current Year
PHP,13934,25.9,Current Year
TypeScript,13711,25.5,Current Year
C++,12705,23.7,Current Year


## Databases

In [0]:
YearComp("DatabaseWorkedWith","DatabaseDesireNextYear").withColumnRenamed("DatabaseWorkedWith","Database").display()

Database,count,Percent,Year
MySQL,23355,55.8,Current Year
PostgreSQL,15543,37.1,Current Year
Microsoft SQL Server,14070,33.6,Current Year
SQLite,13233,31.6,Current Year
MongoDB,11297,27.0,Current Year
Redis,8020,19.1,Current Year
MariaDB,7329,17.5,Current Year
Oracle,6958,16.6,Current Year
Firebase,6215,14.8,Current Year
Elasticsearch,5948,14.2,Current Year


## Frameworks, libraries

In [0]:
YearComp("MiscTechWorkedWith","MiscTechDesireNextYear").withColumnRenamed("MiscTechWorkedWith","Misc").display()

Misc,count,Percent,Year
Node.js,18258,52.1,Current Year
.NET,12599,35.9,Current Year
.NET Core,9724,27.7,Current Year
Pandas,5652,16.1,Current Year
React Native,4087,11.7,Current Year
TensorFlow,4082,11.6,Current Year
Unity 3D,3826,10.9,Current Year
Ansible,2549,7.3,Current Year
Flutter,2557,7.3,Current Year
Keras,2275,6.5,Current Year


## Collaboration technologies

In [0]:
YearComp("NEWCollabToolsWorkedWith","NEWCollabToolsDesireNextYear").withColumnRenamed("NEWCollabToolsWorkedWith","Colaboration Tool").display()

Colaboration Tool,count,Percent,Year
Github,38242,83.3,Current Year
Slack,24576,53.5,Current Year
Jira,21998,47.9,Current Year
"Google Suite (Docs, Meet, etc)",19394,42.2,Current Year
Gitlab,17182,37.4,Current Year
Confluence,15012,32.7,Current Year
Trello,13860,30.2,Current Year
Microsoft Teams,11731,25.6,Current Year
Microsoft Azure,6860,14.9,Current Year
Stack Overflow for Teams,2518,5.5,Current Year


## Platforms

In [0]:
YearComp("PlatformWorkedWith","PlatformDesireNextYear").withColumnRenamed("PlatformWorkedWith","Platform").display()

Platform,count,Percent,Year
Linux,27386,55.9,Current Year
Windows,25995,53.1,Current Year
Docker,17421,35.6,Current Year
AWS,13196,26.9,Current Year
Android,12693,25.9,Current Year
MacOS,11691,23.9,Current Year
Raspberry Pi,7335,15.0,Current Year
Microsoft Azure,7179,14.7,Current Year
WordPress,6907,14.1,Current Year
Google Cloud Platform,6858,14.0,Current Year


## Web Frameworks

In [0]:
YearComp("WebframeWorkedWith","WebframeDesireNextYear").withColumnRenamed("WebframeWorkedWith","Web Framework").display()

Web Framework,count,Percent,Year
jQuery,16164,44.7,Current Year
React.js,13230,36.6,Current Year
Angular,9379,25.9,Current Year
ASP.NET,8200,22.7,Current Year
Express,8073,22.3,Current Year
ASP.NET Core,7222,20.0,Current Year
Vue.js,6499,18.0,Current Year
Spring,6014,16.6,Current Year
Angular.js,5978,16.5,Current Year
Django,5122,14.2,Current Year


#Labor market
## Average annual salary by years of experiance.

In [0]:
%sql
select YearStampCodePro as Proffesional_experience, round(avg(ConvertedComp),2) as Average_Annual_Salary_USD
from so_joined
where ConvertedComp<>'NA' and ConvertedComp>0 and YearsCodePro<>'NA'
group by YearStampCodePro
order by cast(substring(trim(YearStampCodePro),1,2) as int)

Proffesional_experience,Average_Annual_Salary_USD
0 - 1 year,46117.07
1 - 5 years,61953.23
6 - 10 years,86972.93
11 - 20 years,107972.32
21 - 30 years,120473.97
31 - 50 years,128614.53
51 or more years,125787.15


##Satisfaction with the current job

In [0]:
%sql
select JobSat as Job_Satisfaction, count(JobSat) as cnt
from so_joined
where JobSat<>'NA'
group by JobSat
order by cnt desc

Job_Satisfaction,cnt
Very satisfied,14611
Slightly satisfied,13930
Slightly dissatisfied,7124
Neither satisfied nor dissatisfied,5783
Very dissatisfied,3746


## Job search

In [0]:
%sql
select JobSeek as Job_Seek_Status, count(JobSat) as cnt
from so_joined
where JobSeek<>'NA'
group by JobSeek
order by cnt desc

Job_Seek_Status,cnt
"I’m not actively looking, but I am open to new opportunities",29776
I am not interested in new job opportunities,13001
I am actively looking for a job,8950


## Important factors at work

In [0]:
ArrColPercent("JobFactors").drop(col("count")).display()

JobFactors,Percent
"Languages, frameworks, and other technologies I’d be working with",51.3
Office environment or company culture,44.5
Flex time or a flexible schedule,43.9
Opportunities for professional development,41.4
Remote work options,33.3
How widely used or impactful my work output would be,20.8
Industry that I’d be working in,15.3
Family friendliness,12.1
Financial performance or funding status of the company or organization,11.9
Specific department or team I’d be working on,11.8


## Influence on technology purchases

In [0]:
%sql
select PurchaseWhat as Influence_on_technology_purchases, count(PurchaseWhat) as cnt
from so_joined
where PurchaseWhat<>'NA'
group by PurchaseWhat
order by cnt desc

Influence_on_technology_purchases,cnt
I have little or no influence,16987
I have some influence,15379
I have a great deal of influence,6998


## Company size

In [0]:
%sql
select OrgSize as Organisation_size, count(OrgSize) as cnt
from so_joined
where OrgSize<>'NA'
group by OrgSize
order by cast(substring(replace(OrgSize,',',''),1,charindex(' ',replace(OrgSize,',',''))) as int) asc

Organisation_size,cnt
"Just me - I am a freelancer, sole proprietor, etc.",2190
2 to 9 employees,4409
10 to 19 employees,4127
20 to 99 employees,9597
100 to 499 employees,8311
500 to 999 employees,2880
"1,000 to 4,999 employees",4863
"5,000 to 9,999 employees",1808
"10,000 or more employees",6149


##Overtime frequency

In [0]:
%sql
select NEWOvertime as Overtime_frequency, count(NEWOvertime) as cnt
from so_joined
where NEWOvertime<>'NA'
group by NEWOvertime
order by cnt desc

Overtime_frequency,cnt
Sometimes: 1-2 days per month but less than weekly,11547
Often: 1-2 days per week or more,11027
Occasionally: 1-2 days per quarter but less than monthly,9468
Rarely: 1-2 days per year or less,6471
Never,4718


## Quality of onboarding

In [0]:
%sql
select 
case 
when NEWOnboardGood= "Onboarding? What onboarding?" then "There is no onboarding process in my company"
else NEWOnboardGood
end as Is_Onboarding_Good, 
count(NEWOnboardGood) as cnt
from so_joined
where NEWOnboardGood<>'NA'
group by NEWOnboardGood
order by cnt desc

Is_Onboarding_Good,cnt
Yes,20729
No,12655
There is no onboarding process in my company,9239
